In [3]:
import numpy as np
import os
# 假设的最大长度
max_length = 183
# 假设的氨基酸到one-hot编码的映射字典
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_onehot = {aa: np.eye(len(amino_acids))[i] for i, aa in enumerate(amino_acids)}
def load_embeddings(folder_path, max_length):
    """加载并处理特征文件夹中的所有.npy文件。"""
    features_dict = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.npy'):
            protein_id = filename.split('_')[0]  # 根据文件命名约定提取ID
            feature = np.load(os.path.join(folder_path, filename))
            squeezed_feature = np.squeeze(feature)
            # 处理长度不一致的情况
            if squeezed_feature.shape[0] > max_length:
                padded_feature = squeezed_feature[:max_length, :]
            else:
                padding = np.zeros((max_length - squeezed_feature.shape[0], squeezed_feature.shape[1]))
                padded_feature = np.vstack([squeezed_feature, padding])
            features_dict[protein_id] = padded_feature
    return features_dict

def combine_features(esm_folder, bert_folder, data, max_length):
    esm_features = load_embeddings(esm_folder, max_length)
    bert_features = load_embeddings(bert_folder, max_length)
    combined_features = {}
    labels = []
    for label, seq, class_label in data:
        if label in esm_features and label in bert_features:
            one_hot_encoded = np.array([aa_to_onehot.get(aa, np.zeros(len(amino_acids))) for aa in seq])
            padding_length = max_length - len(one_hot_encoded)
            one_hot_padded = np.pad(one_hot_encoded, ((0, padding_length), (0, 0)), 'constant')
            
            esm_embedding = esm_features[label]
            bert_embedding = bert_features[label]
            combined_feature = np.concatenate([bert_embedding, one_hot_padded, esm_embedding], axis=1)
            combined_features[label] = combined_feature
            labels.append(class_label)
    return combined_features, labels
# 示例路径和数据
esm_folder = '/Workspace/xinxinpeng/ZeHua/progect1/final_test/training_dataset/train_esm_embeddings'
bert_folder = '/Workspace/xinxinpeng/ZeHua/progect1/final_test/training_dataset/train_bertembedding'
def load_sequence_data_from_fasta(filepath):
    """从FASTA文件加载序列数据，返回包含（序列ID, 序列, 标签）的列表"""
    with open(filepath, 'r') as file:
        lines = file.readlines()

    data = []
    for i in range(0, len(lines), 2):
        header = lines[i].strip()
        sequence = lines[i+1].strip()
        # 假设头部信息是这样的格式：>AP02125|1
        parts = header[1:].split('|')  # 移除'>', 分割得到 ['AP02125', '1']
        if len(parts) == 2:
            seq_id, label = parts
            data.append((seq_id, sequence, int(label)))  # 将标签转换为整数，如果需要的话
        else:
            print(f"Error parsing header: {header}")
    return data
fasta_file_path = '/Workspace/xinxinpeng/ZeHua/progect1/final_test/training_dataset/corrected_fasta.fasta'
data = load_sequence_data_from_fasta(fasta_file_path)
train_combined_features_dict, train_labels = combine_features(esm_folder, bert_folder, data, max_length)

In [4]:
train_combined_features_dict, train_labels = combine_features(esm_folder, bert_folder, data, max_length)

KeyboardInterrupt: 

In [5]:
from torch_geometric.data import DataLoader, Data
import torch

def prepare_dataset(features_dict, labels):
    dataset = []
    for key, features in features_dict.items():
        # 假设 `labels` 列表的顺序与 `features_dict` 中的键对应
        index = list(features_dict.keys()).index(key)  # 获取当前键的索引
        label = torch.tensor([labels[index]], dtype=torch.float32)  # 使用索引从列表获取标签
        features_tensor = torch.tensor(features, dtype=torch.float32)
        data = Data(x=features_tensor, y=label)
        dataset.append(data)
    return dataset


# 准备训练数据
train_dataset = prepare_dataset(train_combined_features_dict, train_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


/Workspace/xinxinpeng/miniconda3/envs/evescape/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Workspace/xinxinpeng/miniconda3/envs/evescape/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
import numpy as np
import os

# 假设的最大长度
max_length = 183
# 假设的氨基酸到one-hot编码的映射字典
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_onehot = {aa: np.eye(len(amino_acids))[i] for i, aa in enumerate(amino_acids)}

def load_embeddings(folder_path, max_length):
    """加载并处理特征文件夹中的所有.npy文件。"""
    features_dict = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.npy'):
            protein_id = filename.split('|')[0]  # 根据文件命名约定提取ID
            feature = np.load(os.path.join(folder_path, filename))
            squeezed_feature = np.squeeze(feature)
            # 处理长度不一致的情况
            if squeezed_feature.shape[0] > max_length:
                padded_feature = squeezed_feature[:max_length, :]
            else:
                padding = np.zeros((max_length - squeezed_feature.shape[0], squeezed_feature.shape[1]))
                padded_feature = np.vstack([squeezed_feature, padding])
            features_dict[protein_id] = padded_feature
    return features_dict

def combine_features(esm_folder, bert_folder, data, max_length):
    esm_features = load_embeddings(esm_folder, max_length)
    bert_features = load_embeddings(bert_folder, max_length)
    combined_features = {}
    labels = []

    for label, seq, class_label in data:
        if label in esm_features and label in bert_features:
            one_hot_encoded = np.array([aa_to_onehot.get(aa, np.zeros(len(amino_acids))) for aa in seq])
            padding_length = max_length - len(one_hot_encoded)
            one_hot_padded = np.pad(one_hot_encoded, ((0, padding_length), (0, 0)), 'constant')
            
            esm_embedding = esm_features[label]
            bert_embedding = bert_features[label]
            combined_feature = np.concatenate([bert_embedding, one_hot_padded, esm_embedding], axis=1)
            combined_features[label] = combined_feature
            labels.append(class_label)

    return combined_features, labels
# 示例路径和数据
esm_folder = '/Workspace/xinxinpeng/ZeHua/progect1/final_test/test_dataset/test_esm_embeddings'
bert_folder = '/Workspace/xinxinpeng/ZeHua/progect1/final_test/test_dataset/test_bert_embeddings'
def load_sequence_data_from_fasta(filepath):
    """从FASTA文件加载序列数据，返回包含（序列ID, 序列, 标签）的列表"""
    with open(filepath, 'r') as file:
        lines = file.readlines()

    data = []
    for i in range(0, len(lines), 2):
        header = lines[i].strip()
        sequence = lines[i+1].strip()
        # 假设头部信息是这样的格式：>AP02125|1
        parts = header[1:].split('|')  # 移除'>', 分割得到 ['AP02125', '1']
        if len(parts) == 2:
            seq_id, label = parts
            data.append((seq_id, sequence, int(label)))  # 将标签转换为整数，如果需要的话
        else:
            print(f"Error parsing header: {header}")

    return data
fasta_file_path = '/Workspace/xinxinpeng/ZeHua/progect1/final_test/test_dataset/test_dataset.fasta'
data = load_sequence_data_from_fasta(fasta_file_path)
test_combined_features_dict, test_labels = combine_features(esm_folder, bert_folder, data, max_length)

In [7]:
test_dataset = prepare_dataset(test_combined_features_dict, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [8]:
train_dataset

[Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 3604], y=[1]),
 Data(x=[183, 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN1D(nn.Module):
    def __init__(self, num_features):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        # 根据上面的卷积和池化计算出的尺寸更新下面的尺寸
        self.flattened_size = 64 * ((183 // 2) // 2)  # 183是序列长度，每个池化层之后长度减半
        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 3604, 183)  # 确保尺寸是(batch_size, channels, length)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = x.view(-1, self.flattened_size)  # Flatten all dimensions except batch
        x = self.dropout(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return torch.sigmoid(x)



In [10]:
# 在数据加载部分处理
# 保留原始的列表索引方式，确保顺序一致性
def prepare_data_for_cnn(features_dict, labels_list):
    dataset = []
    keys = list(features_dict.keys())
    for i, key in enumerate(keys):
        features = features_dict[key]
        features_tensor = torch.tensor(features, dtype=torch.float32).transpose(0, 1)
        label = torch.tensor([labels_list[i]], dtype=torch.float32)
        dataset.append(Data(x=features_tensor, y=label))
    return dataset

# 确保 train_labels 是按照 features_dict 中的键顺序排列的
dataset = prepare_data_for_cnn(train_combined_features_dict, train_labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
# 假设 test_combined_features_dict 和 test_labels 已经定义
test_dataset = prepare_data_for_cnn(test_combined_features_dict, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [11]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, matthews_corrcoef

def train(model, train_loader, test_loader, optimizer, criterion, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for data in train_loader:
            optimizer.zero_grad()
            output = model(data.x.float())
            loss = criterion(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        print(f'Epoch {epoch+1}: Train Loss = {total_train_loss / len(train_loader)}')
        
        model.eval()
        total_test_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for data in test_loader:
                output = model(data.x.float())
                loss = criterion(output, data.y.view(-1, 1).float())
                total_test_loss += loss.item()
                # Use sigmoid threshold to determine binary predictions
                predictions = (output > 0.5).float()
                all_preds.extend(predictions.view(-1).cpu().numpy())
                all_labels.extend(data.y.view(-1, 1).cpu().numpy())

        # Calculating metrics
        average_test_loss = total_test_loss / len(test_loader)
        acc = accuracy_score(all_labels, all_preds)
        try:
            auc = roc_auc_score(all_labels, all_preds)
        except ValueError:
            auc = float('nan')  # In case there is only one class present in y_true
        cm = confusion_matrix(all_labels, all_preds)
        sens = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if cm.shape[0] > 1 else 0
        spec = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if cm.shape[0] > 1 else 0
        mcc = matthews_corrcoef(all_labels, all_preds)

        print(f'Test Loss = {average_test_loss:.4f}')
        print(f'ACC = {acc:.4f}, AUC = {auc:.4f}, SENS = {sens:.4f}, SPEC = {spec:.4f}, MCC = {mcc:.4f}')



In [13]:
import torch.optim as optim  # 导入优化器模块
import torch.nn as nn  # 导入神经网络模块
model = CNN1D(num_features=3604)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

train(model, train_loader,test_loader, optimizer, criterion, num_epochs=20)

/Workspace/xinxinpeng/miniconda3/envs/evescape/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1: Train Loss = 0.20676081602155053
Test Loss = 2.4190
ACC = 0.5911, AUC = 0.5911, SENS = 0.6146, SPEC = 0.5677, MCC = 0.1825
Epoch 2: Train Loss = 0.13940491621029674
Test Loss = 4.9494
ACC = 0.5524, AUC = 0.5524, SENS = 0.6829, SPEC = 0.4219, MCC = 0.1086
Epoch 3: Train Loss = 0.12912263975461355
Test Loss = 4.5041
ACC = 0.5596, AUC = 0.5596, SENS = 0.7051, SPEC = 0.4141, MCC = 0.1245
Epoch 4: Train Loss = 0.1140720003909252
Test Loss = 10.6922
ACC = 0.5085, AUC = 0.5085, SENS = 0.7428, SPEC = 0.2741, MCC = 0.0192
Epoch 5: Train Loss = 0.10079961419915376
Test Loss = 6.3187
ACC = 0.5824, AUC = 0.5824, SENS = 0.6257, SPEC = 0.5391, MCC = 0.1653
Epoch 6: Train Loss = 0.0804392508207965
Test Loss = 6.8948
ACC = 0.5560, AUC = 0.5560, SENS = 0.7461, SPEC = 0.3659, MCC = 0.1211
Epoch 7: Train Loss = 0.06942095240123963
Test Loss = 11.8637
ACC = 0.5638, AUC = 0.5638, SENS = 0.7695, SPEC = 0.3581, MCC = 0.1400
Epoch 8: Train Loss = 0.07557616140859734
Test Loss = 8.6676
ACC = 0.5693, A